<a href="https://colab.research.google.com/github/rabimba/GDE-ML-Artifacts/blob/main/Gemini_Function_Calling_Example_with_Array_Outputs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gemini Function Calling Example with Array Outputs

In [ ]:
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

In [ ]:
import vertexai
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

PROJECT_ID = "gde-all-rk"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
)

In [ ]:
model = GenerativeModel("gemini-1.5-pro")

Consider that we have the following unstructured text data:

In [ ]:
financial_data = """*** Starlight Cinema   Date: 01/05/2024***
Movie: Strange World
Admissions: 155   Daily Revenue: $2150.00

Theater Total Admissions: 9876
Total Revenue :$138900.55

*****
The Flicker House  1/3/24
Space Odyssey 2.0 (Re-Release)
1001 Broadway, Big City, NY
Daily Admiss: 85
Daily Revenue:$1235

Theater Total Admissions 6450
Theater Total Revenue $92433.00

*** Discount Movies! ***
Bargain Bin Cinema
Movie Title: Critters Attack!
Date: 1/6/24
Daily Total Adms:  205
Daily Rev: $999.99
Theater Total: Admissions 15432  Revenue: $76432.10

***
Majestic Movies
Date:2024/01/04   Movie: The Robot Uprising
Daily Admissions - 321  Revenue: $4756.00

Running Total Admissions: 25589
Running Total Revenue:$365432

***PLEX-A-RAMA***
Movie: Mystery Island Adventure
1/5/2024
Daily Admissions: 140
Daily Revenue:$1980.50
Theater Totals Not Available
"""

We can define a function call to get a structured object back from unstructured receipt data:

In [ ]:
get_receipt_data = FunctionDeclaration(
    name="get_receipt_data",
    description="Extract financical data from box office receipts",
    parameters={
        "type": "object",
        "properties": {
            "movie": {"type": "string", "description": "Movie Name"},
              "theater": {"type": "string", "description": "Theater Name"},
                "date": {"type": "string", "description": "Date of the event"},
                "daily_total_admissions": {"type": "number", "description": "Daily Total Admissions",},
                "daily_total_revenue": {"type": "number", "description": "Daily Total Revenue",},
              "theater_total_admissions": {"type": "number", "description": "Theater Total Admissions",},
              "theater_total_revenue": {"type" : "number", "description": "Theater Total Revenue",},
        },
    },
)

receipt_data_tool = Tool(
    function_declarations=[get_receipt_data],
)

In [ ]:
response = model.generate_content(
    financial_data,
    tools=[receipt_data_tool],
)

In [ ]:
response.candidates[0].content.parts[0]

text: "I cannot fulfill this request. The available tools lack the functionality to calculate average daily revenue."

However, we only get back one of the records.

We can instead define an array output in the `FunctionDeclaration` as in:

In [ ]:
get_receipt_data = FunctionDeclaration(
    name="get_receipt_data",
    description="Extract the financial data from box office receipts",
    parameters={
        "type": "object",
        "properties": {
            "records": {
                "type": "array",
                "description": "A list of receipt records",
                "items": {
                    "description": "Financial data for a single receipt",
                    "type": "object",
                    "properties": {
                        "movie": {"type": "string", "description": "Movie Name"},
                        "theater": {"type": "string", "description": "Theater Name"},
                        "date": {"type": "string", "description": "Date of the event"},
                        "daily_total_admissions": {"type": "number", "description": "Daily Total Admissions"},
                        "daily_total_revenue": {"type": "number", "description": "Daily Total Revenue"},
                        "theater_total_admissions": {"type": "number", "description": "Theater Total Admissions"},
                        "theater_total_revenue": {"type": "number", "description": "Theater Total Revenue"},
                    },
                },
            },
        },
    },
)

receipt_data_tool = Tool(
    function_declarations=[get_receipt_data]
)

In [ ]:
response = model.generate_content(
    financial_data,
    tools=[receipt_data_tool],
)

In [ ]:
response.candidates[0].content.parts[0]

text: "I cannot fulfill this request. The available tools lack the functionality to extract the theater totals from the provided data."